<a href="https://colab.research.google.com/github/johri002/Techtinium-Python-assignment/blob/master/Techtinium_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimum Cost resource alocator
### using unbounded knapsack approach

Problem Statement:
The following are the available machines along with their respective capacities.

    Large – 10 units
    XLarge – 20 units 
    2XLarge – 40 units
    4XLarge – 80 units
    8XLarge – 160 units
    10XLarge – 320 units

These machines are located all over the world and depending on which part of the world they are located in, they have associated costs based on the number of hours they are utilised. Below are the per hour costs
```
 	     New York  India  China
Large	  $120	 $140   $110
XLarge     $230	        $200
2XLarge	$450	 $413	 
4XLarge	$774	 $890   $670
8XLarge	$1400	$1300  $1180
10XLarge   $2820	$2970	 
```

Write an optimized resource allocator program that can be used for cost planning. The program takes the below 2 inputs:

Hours: No of hours the machine is required to run
Capacity: No of units are required (Will always be multiple of 10)

Based on these inputs, the program should optimally allocate the resources such that the cost of the capacity required is minimum. Calculate this for every region and show them as the below example.


**Sample Input:** 
Capacity of 1150 units for 1 Hour

 
**Expected Output:**
```
{
  “Output”: [
    {
      “region”: “New York”,
      “total_cost”: “$10150”,
      “machines”: [
        (8XLarge, 7),
        (XLarge, 1),
        (Large, 1)
      ]
    },
    {
      “region”: “India”,
      “total_cost”: “$9520”,
      “machines”: [
        (8XLarge, 7),
        (Large, 3)
      ] 
    },
    {
      “region”: “China”,
      “total_cost”: “$8570”,
      “machines”: [
        (8XLarge, 7),
        (XLarge, 1),
        (Large, 1)
      ] 
  },
]
}
```
 
**Other Sample Inputs include:**
```
        230 units for 5 Hours
        100 units for 24 Hours
        1100 units for 12 Hours
```

In [8]:
import numpy as np

INF = 1000000

def data_generation(machine_type, time, machines, country_prices):
  new_machine_type=[]
  new_machines=[]
  new_country_prices=[]  
  for i in range(len(machines)):
    if country_prices[i] != None:
      new_machine_type.append(machine_type[i])      
      new_machines.append(machines[i])      
      new_country_prices.append(country_prices[i] * time)

  return new_machine_type, new_machines, new_country_prices      


def minimum_cost_resource_allocator(country, machine_type, machines, country_prices, units):

  n = len(country_prices)
  # Solution matrix initialization
  solution_matrix = [[None for j in range(units+1)] for i in range(n+1)] 
  for i in range(0, n+1):
    for j in range(units+1):
      # Base cases
      if j==0 and i>0:
        solution_matrix[i][j]=0

      elif i==0:  
        solution_matrix[i][j]=INF

  for i in range(1, n+1):
    for j in range(1, units+1):

      if machines[i-1]<=j:
        solution_matrix[i][j] = min( country_prices[i-1] + solution_matrix[i][j-machines[i-1]], 
                                    solution_matrix[i-1][j])
      else:
        solution_matrix[i][j] = solution_matrix[i-1][j]

  lowest_weight = solution_matrix[n][units]
  machines_with_qty = find_machines(solution_matrix, n, units)
  
  output = {
  "region": country,
  "total_cost": "$"+str(lowest_weight),
  "machines": list(zip(machines_with_qty.keys(), machines_with_qty.values())) 
  }
  return output

def find_machines(dp, n, units):
  x = n
  y = units
  hash = dict()
  while (x > 0 and y > 0):
    if dp[x][y] == dp[x - 1][y]:
        x-=1
    elif (dp[x - 1][y] == dp[x][y - machines[x - 1]] + country_prices[x - 1]): 
        x-=1
    else:
      if machine_type[x - 1] in hash.keys():
        hash[machine_type[x - 1]] += 1
      else:
        hash[machine_type[x - 1]] = 1      
      # print("including wt " + str(machines[x - 1]) + " with value " + str(country_prices[x - 1]))
      y -= machines[x - 1]
  return hash

  
if __name__ == "__main__":
  units = int(input("Capacity (units): "))
  time = int(input("Operation time (hours): "))
  
  countries= ["NewYork", "India", "China"]
  NewYork  = [120,230,450,774,1400,2820]
  India    = [140,None,413,890,1300,2970]		
  China    = [110,200,None,670,1180,None]
  allocated_resource=[]

  for country in countries:
    machine_type = ['Large', 'XLarge', '2XLarge', '4XLarge', '8XLarge', '10XLarge']
    machines = [10, 20, 40, 80, 160, 320]	
    country_pricelist = [NewYork, India, China]
    country_and_price = dict(zip(countries, country_pricelist))

    machine_type, machines, country_prices = data_generation(machine_type, time, machines, country_and_price[country])
    allocated_resource.append( minimum_cost_resource_allocator(country, machine_type, machines, country_prices, units) )
  
  print({
      "Output" : allocated_resource
      })

Capacity (units): 1150
Operation time (hours): 1
{'Output': [{'region': 'NewYork', 'total_cost': '$10150', 'machines': [('8XLarge', 7), ('XLarge', 1), ('Large', 1)]}, {'region': 'India', 'total_cost': '$9520', 'machines': [('8XLarge', 7), ('Large', 3)]}, {'region': 'China', 'total_cost': '$8570', 'machines': [('8XLarge', 7), ('XLarge', 1), ('Large', 1)]}]}
